In [36]:
import os
import pandas as pd
import pandasgui as pdgui
import numpy as np
import matplotlib.pyplot as plt

# CSV Reader

Here we read the CSV file

In [37]:
student_data = {
    "bio": pd.read_csv("bio.csv"),
    "net": pd.read_csv("net.csv"),
    "gate": pd.read_csv("gate.csv"),
    "tenth": pd.read_csv("tenth.csv"),
    "others": pd.read_csv("others.csv"),
    "masters": pd.read_csv("masters.csv"),
    "plustwo": pd.read_csv("plustwo.csv"),
    "bachelors": pd.read_csv("bachelors.csv"),
    "experience": pd.read_csv("experience.csv"),
}

global reason

reason = pd.DataFrame(columns = ['ReasonID', 'ReasonDesc'])
for i in range(49):
    reason.loc[i] = [i,"NA"]

"""
Avoids NaN by creating different alternatives, good for everyone,
keep adding as you see fit
"""

for key in student_data.keys():
    if "Institute" in student_data[key].keys():
        student_data[key]["Institute"].fillna("NA",inplace=True) #Replace all NaN values with "NA"
    if "Programme" in student_data[key].keys():
        student_data[key]["Programme"].fillna("NA",inplace=True) #Replace all NaN values with "NA"
    if "Specialization" in student_data[key].keys():
        student_data[key]["Specialization"].fillna("NA",inplace=True) #Replace all NaN values with "NA"
    if "Discipline" in student_data[key].keys():
        student_data[key]["Discipline"].fillna("NA",inplace=True) #Replace all NaN values with "NA"
    if "Percentage" in student_data[key].keys():
            student_data[key]["Percentage"].fillna(student_data[key]["CGPA"]*9.5,inplace=True) #Replace all NaN values with "NA"

# Functions Block

Here we store all the functions to work with, some apt, some stupid.

### `checkIIT()`

### `checkNET()`

### `checkGATE()`

### `reason()`

### `appendShortListed()`

In [38]:

def checkIIT(Data,CGPCri):
    """
    This Function checks for IITians, just a query check that's all
    SHORTLISTING IIT CANDIDATES WITH COOL CG
    """
    mylist =[]
    for i,row in Data.iterrows():
        
        if(
            "iit" in row["Institute"].lower() or
            "indian institute of technology" in row["Institute"].lower()
        ):
            if (
                np.isnan(row["CGPA"]) or
                row["CGPA"] < CGPCri
            ):
                continue
            else:
                mylist.append(i)
                reason = Reason(reason,i,5)
    return mylist #Return list of indices

In [39]:
def Reason(i,id):
    """
    The Function takes in id for reason and puts it in reason csv file
    ID For Reason Are:
    0: Qualified via GATE
    1: Not Enough 10/+2 Score
    2: Not Enough GATE Score
    3: Didn't Qualify GATE
    4: Qualified via NET-UGC
    5: Qualified as Graduates/Masters of IIT
    NET-UGC and Graduate/Post-Grad in IIT are considered Exceptions
    and not fullfilling these cannot be listed as reason
    """
    reason.loc[i]["ReasonID"] = id
    
    if id == 0:
        reason.loc[i]["ReasonDesc"] = "Qualified via GATE"
    elif id == 1:
        reason.loc[i]["ReasonDesc"] = "Not Enough 10/+2 Score"
    elif id == 2:
        reason.loc[i]["ReasonDesc"] = "Not Enough GATE Score"
    elif id == 3:
        reason.loc[i]["ReasonDesc"] = "Didn't Qualify GATE"
    elif id == 4:
        reason.loc[i]["ReasonDesc"] = "Qualified via NET-UGC"
    elif id == 5:
        reason.loc[i]["ReasonDesc"] = "Qualified as Graduates/Masters of IIT"
    elif id == 6:
        reason.loc[i]["ReasonDesc"] = "Not Enough Bachelors Score"
    elif id == 7:
        reason.loc[i]["ReasonDesc"] = "Not Enough Masters Score"
    else:
        reason.loc[i]["ReasonDesc"] = "NA"


In [40]:

def checkNET(Data):
    global reason
    """
    This function Checks for NET-UGC Qualification
    """
    global reason
    mylist = []
    for i,row in Data.iterrows():
        if row["Discipline"] != "NA":
            mylist.append(i)
            Reason(i,4)
    return mylist


In [41]:

def checkGATE(MyDict):
    """
    This function checks for GATE Qualification
    if gate qualified, check masters
    if masters yes, exec masters block
    else exec bachelors block

    All CGPA have been converted to percentage if not present
    so all evaluations will be done on basis of percentage only
    """
    global reason
    selected = True #Arbitrary var to hold selection logic,
                    #Helps avoid loops and stuff
    cutoff = 0 #Gate Marks
    perc = 0 #Percentage
    category = "GEN" #Default
    mylist = []
    for i,row in MyDict["gate"].iterrows():
        if row["Discipline"] != "NA":
            if row["Score"] > 302: #Least qualifying score possible
                selected = True
            else:
                Reason(i,2) #2: Not Enough Gate Score
                selected = False
                continue    #Eliminate some BS by skipping beforehand
        else:
            Reason(i,3) #3: Didnt qualify gate
            selected = False
            continue
        if MyDict["masters"].iloc[i]["Programme"] == "NA":
            #Btech Part
            cutoff = 500
            perc = 70
            category =  MyDict["bio"].iloc[i]["Category"]
            #category relaxations
            if category == "OBC":
                cutoff = 0.9 * cutoff
            elif category in ["SC","ST","PWD"]:
                cutoff = 0.67 * cutoff
                perc = perc - 5
            #GATE Filtering
            if row["Score"] < cutoff:
                #Check GATE Score
                Reason(i,2) #2: Not enough gate score
                selected = False
                continue
            # Boards Filtering with single relaxation
            if (
                (MyDict["tenth"].iloc[i]["Percentage"] < perc and MyDict["plustwo"].iloc[i]["Percentage"] < perc - 10) or
                (MyDict["tenth"].iloc[i]["Percentage"] < perc - 10 and MyDict["plustwo"].iloc[i]["Percentage"] < perc)
            ):
                Reason(i,1) #1: Not qualified via +2/10 score
                selected = False
                continue
            # Bachelors Filtering
            if MyDict["bachelors"].iloc[i]["Percentage"] < perc:
                Reason(i,6) #6: Not enough bachelor score
                selected = False
                continue

        else:
            #Mtech Part
            cutoff = 450
            perc = 60
            category =  MyDict["bio"].iloc[i]["Category"]
            #category relaxations
            if category == "OBC":
                cutoff = 0.9 * cutoff
            elif category in ["SC","ST","PWD"]:
                cutoff = 0.67 * cutoff
                perc = perc - 5
            #GATE Filtering
            if row["Score"] < cutoff:
                #Check GATE Score
                Reason(i,2) #2: Not enough gate score
                selected = False
                continue
            # Boards Filtering with single relaxation
            if (
                (MyDict["tenth"].iloc[i]["Percentage"] < perc and MyDict["plustwo"].iloc[i]["Percentage"] < perc - 10) or
                (MyDict["tenth"].iloc[i]["Percentage"] < perc - 10 and MyDict["plustwo"].iloc[i]["Percentage"] < perc)
            ):
                Reason(i,1) #1: Not qualified via +2/10 score
                selected = False
                continue
            # Bachelors Filtering
            if MyDict["bachelors"].iloc[i]["Percentage"] < perc:
                Reason(i,6) #6: Not enough bachelor score
                selected = False
                continue
            if MyDict["masters"].iloc[i]["Percentage"] < perc:
                Reason(i,7) #6: Not enough master score
                selected = False
                continue
        if selected:
            Reason(i,0) #0: Qualified Via GATE
            mylist.append(i)
    return mylist

In [42]:

def appendShortlisted(SourceDict,ShortList):
    """
    Appends Shortlisted Rows onto dict
    Takes element of id present in ShortList present in SourceDict
    and Puts it in TargetDict
    """
    TargetDict = {}
    for key in SourceDict.keys():
        TargetDict[key] = pd.DataFrame(columns = SourceDict[key].columns.tolist()) #copy column headers
        for i in ShortList:
            TargetDict[key] = TargetDict[key].append( SourceDict[key].iloc[i] )
    return TargetDict

# Algorithm Block

Here we run the actual algorithm

In [45]:
#Create All Lists and Merge Them

selectGATE = checkGATE(student_data)
BTechIIT = checkIIT(student_data["bachelors"],8.0)
MTechIIT = checkIIT(student_data["masters"],8.5)
selectNET = checkNET(student_data["net"])

finalList = list(
    set(selectGATE) | set(BTechIIT) | set(MTechIIT) | set(selectNET)
)

print("ID of Finalists : ",finalList)

finalDF = appendShortlisted(student_data,finalList)


ID of Finalists :  [2, 5, 6, 9, 10, 14, 16, 21, 26, 27, 32, 33, 37, 38, 39, 41, 43, 45, 46]


In [47]:
reason

,ReasonID,ReasonDesc
0,2,Not Enough GATE Score
1,2,Not Enough GATE Score
2,4,Qualified via NET-UGC
3,2,Not Enough GATE Score
4,2,Not Enough GATE Score
5,4,Qualified via NET-UGC
6,4,Qualified via NET-UGC
7,3,Didn't Qualify GATE
8,7,Not Enough Masters Score
9,0,Qualified via GATE
